In [14]:
import numpy as np
import pandas as pd
import pyesg
from series import invest_series, salary_series, age_series, mortality_sim
from allocation import allocation

# projection inputs
years = 40
periods_per_year = 12
n_scenarios = 10

# demographic inputs
age = 30
sex = 'M'
smoker = 'NS'
ret_age = 65
mi = .99

# fund inputs
fund_value_start = 100000
withdraw_amount = 1000000
salary = 100000
salary_growth = .045
invest_pct = .15
inflation = .02
stock_pct_strt = .80
bond_pct_strt = 1 - stock_pct_strt
five_yr_pct = .55
stock_pct_end = .2
bond_pct_end = 1 - stock_pct_end

# instantiate a new model with the required parameters
stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
bond_model = pyesg.GeometricBrownianMotion(mu=0.06, sigma=0.05)

x0 = 100.0                             # the start value of our process
dt = 1/periods_per_year                # the length of each timestep in years
n_steps = periods_per_year * years     # the number of time steps per scenario
# random_state = 259                   # optional random_state for reproducibility

s_model_results = stock_model.scenarios(x0, dt, n_scenarios, n_steps)
b_model_results = bond_model.scenarios(x0, dt, n_scenarios, n_steps)

# create stock and bond index return arrays. 
stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

stock_array = np.insert(stock_return, 0, fund_value_start * stock_pct_strt, axis=1)
bond_array = np.insert(bond_return, 0, fund_value_start * bond_pct_strt, axis=1)

# the last return value is not used so we add a 1 to the end to return the array to its original length
ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
stock_return = np.append(stock_return, ones_to_append, axis=1)
bond_return = np.append(bond_return, ones_to_append, axis=1)

salary_s = salary_series(salary = salary, salary_growth= salary_growth, periods_per_year=periods_per_year, years=years, age = age, ret_age=ret_age)
invest_s = invest_series(salary_s = salary_s, invest_pct = invest_pct, periods_per_year = periods_per_year, withdraw_amount=withdraw_amount, age = age, ret_age=ret_age)
mortality_s = mortality_sim(age = age, ret_age = ret_age, sex = sex, smoker = smoker, n_scenarios = n_scenarios, mi = mi)
allocation_s = allocation(start_pct = stock_pct_strt,five_yr_pct = five_yr_pct,end_pct = stock_pct_end,age = age,ret_age = ret_age,periods_per_year = periods_per_year,years = years )            
age_s = age_series(age = age, periods_per_year=periods_per_year, years=years)

for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_s, allocation_s['equity'], allocation_s['bond']):
    for i in range(1, len(s)):
        s[i] = s[i-1] * s[i]
        b[i] = b[i-1] * b[i]
        total_fund = s[i] + b[i] + invest_s[i-1]/periods_per_year
        s[i] = total_fund * alloc_s
        b[i] = total_fund * alloc_b

total_fund = stock_array + bond_array
df = pd.DataFrame(total_fund.T)



# with pd.ExcelWriter('Outputs/retirement_calc.xlsx') as writer:
#     df.to_excel(writer, sheet_name='sheet1')

In [15]:
print(type(df))
print(df)



<class 'pandas.core.frame.DataFrame'>
                0             1             2             3             4  \
0    1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05   
1    1.004541e+05  9.711156e+04  9.671150e+04  1.011124e+05  9.406144e+04   
2    1.044603e+05  1.012189e+05  9.523892e+04  1.050154e+05  9.657411e+04   
3    1.036738e+05  1.000196e+05  9.982170e+04  1.067224e+05  9.423965e+04   
4    1.044193e+05  1.001617e+05  1.000821e+05  1.062331e+05  8.849741e+04   
..            ...           ...           ...           ...           ...   
476  2.099188e+06  1.166437e+06  4.655067e+06  2.454416e+06  3.080240e+06   
477  2.083829e+06  1.109262e+06  4.671223e+06  2.323778e+06  3.097403e+06   
478  2.105905e+06  1.108918e+06  4.701452e+06  2.332843e+06  3.277295e+06   
479  2.131448e+06  1.139430e+06  4.530120e+06  2.293034e+06  3.309258e+06   
480  2.177805e+06  1.168104e+06  4.550105e+06  2.278898e+06  3.207213e+06   

                5             6      